<a href="https://colab.research.google.com/github/NehaCh11/Apple-Collector/blob/main/i247623_MSAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from copy import deepcopy

# Constants
GRID_SIZE = 10
FLAGSHIP_HEALTH = 10
SUPPORT_HEALTH = 3
AI_DEPTH = 3  # Depth for Minimax

# Ship Class
class Ship:
    def __init__(self, name, health, x, y):
        self.name = name
        self.health = health
        self.position = (x, y)

    def is_destroyed(self):
        return self.health <= 0

# Player Class
class Player:
    def __init__(self, name, is_ai=False):
        self.name = name
        self.is_ai = is_ai
        self.flagship = Ship(f"{name}_Flagship", FLAGSHIP_HEALTH, 0, 0) if name == 'A' else Ship(f"{name}_Flagship", FLAGSHIP_HEALTH, GRID_SIZE-1, GRID_SIZE-1)
        self.support_ships = [
            Ship(f"{name}_Support1", SUPPORT_HEALTH, 1, 0 if name == 'A' else GRID_SIZE-2),
            Ship(f"{name}_Support2", SUPPORT_HEALTH, 0 if name == 'A' else GRID_SIZE-2, 1),
            Ship(f"{name}_Support3", SUPPORT_HEALTH, 1, 1 if name == 'A' else GRID_SIZE-2)
        ]

    def all_ships(self):
        return [self.flagship] + self.support_ships

# Game Class
class Game:
    def __init__(self):
        self.grid = [["-" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.playerA = Player("A")
        self.playerB = Player("B", is_ai=True)
        self.current_player = self.playerA
        self.update_grid()

    def update_grid(self):
        self.grid = [["-" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        for ship in self.playerA.all_ships() + self.playerB.all_ships():
            if not ship.is_destroyed():
                x, y = ship.position
                self.grid[x][y] = ship.name[0]

    def is_valid_move(self, x, y):
        return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and self.grid[x][y] == "-"

    def move_ship(self, ship, x, y):
        if self.is_valid_move(x, y):
            ship.position = (x, y)
            self.update_grid()
            return True
        return False

    def attack(self, attacker, target):
        if self.is_adjacent(attacker.position, target.position):
            target.health -= 2
            self.update_grid()
            return True
        return False

    def is_adjacent(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1

    def is_game_over(self):
        return self.playerA.flagship.is_destroyed() or self.playerB.flagship.is_destroyed()

    def print_grid(self):
        for row in self.grid:
            print(" ".join(row))
        print()

# Minimax with Alpha-Beta Pruning
def minimax(game, depth, alpha, beta, maximizing_player):
    if depth == 0 or game.is_game_over():
        return evaluate_board(game)

    if maximizing_player:
        max_eval = float('-inf')
        for move in get_all_possible_moves(game, game.playerB):  # AI moves
            new_game = make_move(game, move)
            eval = minimax(new_game, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_all_possible_moves(game, game.playerA):  # Human moves
            new_game = make_move(game, move)
            eval = minimax(new_game, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# ExpectiMinimax (a simple version assuming some randomness in the opponent’s moves)
def expectiminimax(game, depth, maximizing_player):
    if depth == 0 or game.is_game_over():
        return evaluate_board(game)

    if maximizing_player:
        max_eval = float('-inf')
        for move in get_all_possible_moves(game, game.playerB):  # AI moves
            new_game = make_move(game, move)
            eval = expectiminimax(new_game, depth - 1, False)
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        # Instead of minimizing, simulate an expected value based on possible random moves
        total_eval = 0
        moves = get_all_possible_moves(game, game.playerA)
        for move in moves:
            new_game = make_move(game, move)
            total_eval += expectiminimax(new_game, depth - 1, True)
        return total_eval / len(moves) if moves else evaluate_board(game)

def evaluate_board(game):
    return sum(ship.health for ship in game.playerB.all_ships()) - sum(ship.health for ship in game.playerA.all_ships())

def get_all_possible_moves(game, player):
    moves = []
    for ship in player.all_ships():
        x, y = ship.position
        # Move actions
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = x + dx, y + dy
            if game.is_valid_move(new_x, new_y):
                moves.append(('move', ship, new_x, new_y))

        # Attack actions with priority on support ships
        opponent_ships = game.playerA.all_ships() if player == game.playerB else game.playerB.all_ships()
        support_ships = [s for s in opponent_ships if s != (game.playerA.flagship if player == game.playerB else game.playerB.flagship)]
        flagship = game.playerA.flagship if player == game.playerB else game.playerB.flagship

        for target in support_ships + [flagship]:  # Check support ships first
            if not target.is_destroyed() and game.is_adjacent(ship.position, target.position):
                moves.append(('attack', ship, target))
    return moves

def make_move(game, move):
    action, ship, x, y = move if move[0] == 'move' else (move[0], move[1], move[2].position[0], move[2].position[1])
    new_game = deepcopy(game)
    if action == 'move':
        new_game.move_ship(ship, x, y)
    elif action == 'attack':
        new_game.attack(ship, move[2])
    return new_game

# AI Move based on user choice
def ai_turn(game, ai_algorithm):
    best_move = None
    best_value = float('-inf') if ai_algorithm != "expectiminimax" else 0

    for move in get_all_possible_moves(game, game.playerB):
        new_game = make_move(game, move)
        if ai_algorithm == "minimax":
            move_value = minimax(new_game, AI_DEPTH, float('-inf'), float('inf'), False)
        elif ai_algorithm == "alpha-beta":
            move_value = minimax(new_game, AI_DEPTH, float('-inf'), float('inf'), False)
        elif ai_algorithm == "expectiminimax":
            move_value = expectiminimax(new_game, AI_DEPTH, False)

        if move_value > best_value:
            best_value = move_value
            best_move = move

    if best_move:
        action, ship, x, y = best_move if best_move[0] == 'move' else (best_move[0], best_move[1], best_move[2].position[0], best_move[2].position[1])
        if action == 'move':
            game.move_ship(ship, x, y)
            print(f"AI moved {ship.name} to position ({x}, {y}).")
        elif action == 'attack':
            target = best_move[2]
            game.attack(ship, target)
            print(f"AI attacked {target.name} with {ship.name}.")


def play_game():
    # Ask user for AI algorithm
    ai_algorithm = input("Choose AI algorithm (minimax, alpha-beta, expectiminimax): ").lower()
    valid_algorithms = {"minimax", "alpha-beta", "expectiminimax"}
    while ai_algorithm not in valid_algorithms:
        print("Invalid choice. Please select again.")
        ai_algorithm = input("Choose AI algorithm (minimax, alpha-beta, expectiminimax): ").lower()

    game = Game()
    while not game.is_game_over():
        game.print_grid()
        if game.current_player.is_ai:
            ai_turn(game, ai_algorithm)
            print("AI made its move")
        else:
            # Mapping user input to actual ships
            ship_map = {
                "A1": game.playerA.flagship,
                "A2": game.playerA.support_ships[0],
                "A3": game.playerA.support_ships[1],
                "A4": game.playerA.support_ships[2]
            }

            ship_choice = input("Choose your ship (A1, A2, A3, A4): ").upper()
            ship = ship_map.get(ship_choice)

            if ship and not ship.is_destroyed():
                action = input("Choose action (move/attack): ").lower()
                if action == 'move':
                    while True:
                        try:
                            # Prompt for coordinates and validate
                            x, y = map(int, input("Enter new position (x y): ").split())
                            if x < 0 or x >= GRID_SIZE or y < 0 or y >= GRID_SIZE:
                                print("Invalid move! Position is out of bounds. Try again.")
                            elif game.grid[x][y] != "-":
                                print("Invalid move! Position is already occupied. Try again.")
                            elif game.move_ship(ship, x, y):
                                print("Move successful!")
                                break
                            else:
                                print("Invalid move! Try again.")
                        except ValueError:
                            print("Invalid input, please enter two numbers separated by a space.")
                elif action == 'attack':
                    target_choice = input("Choose target (B1, B2, B3, B4): ").upper()
                    target_map = {
                        "B1": game.playerB.flagship,
                        "B2": game.playerB.support_ships[0],
                        "B3": game.playerB.support_ships[1],
                        "B4": game.playerB.support_ships[2]
                    }
                    target = target_map.get(target_choice)

                    # Check if target exists and is within range
                    if target:
                        if game.is_adjacent(ship.position, target.position):
                            if game.attack(ship, target):
                                print("Attack successful!")
                            else:
                                print("Attack failed!")
                        else:
                            print("Target is out of range. Try a closer target.")
                    else:
                        print("Invalid target selection.")
            else:
                print("Invalid ship choice or the ship is destroyed.")

        # Alternate turn
        game.current_player = game.playerA if game.current_player == game.playerB else game.playerB

    print("Game Over")
    if game.playerA.flagship.is_destroyed():
        print("AI Wins!")
    else:
        print("Player Wins!")

play_game()


Choose AI algorithm (minimax, alpha-beta, expectiminimax): expectiminimax
A A - - - - - - - -
A A - - - - - - B -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- B - - - - - - - -
- - - - - - - - - B

Choose your ship (A1, A2, A3, A4): a2
Choose action (move/attack): attack
Choose target (B1, B2, B3, B4): b1
Target is out of range. Try a closer target.
A A - - - - - - - -
A A - - - - - - B -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- B - - - - - - - -
- - - - - - - - - B

AI made its move
A A - - - - - - - -
A A - - - - - - B -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- B - - - - - - - -
- - - - - - - - - B

Choose your ship (A1, A2, A3, A4): a4
Choose action (move/attack): move
Enter new position (x y): 3 3
Move successful!
A A - - - - - - - -
A - - - - - -